In [3]:
import pandas as pd
import asyncio
import aiohttp
import os
import pymysql
import time
import re
from bs4 import BeautifulSoup
from selenium import webdriver
from selenium.webdriver.common.by import By
from datetime import datetime, timedelta

# 날짜 함수
def format_date(date):
    return date.strftime('%Y-%m-%d')

# 시작 날짜와 종료 날짜 설정
start_date = datetime(2024, 3, 31)
end_date = datetime(2024, 3, 31)

# 날짜 형식 변환
formatted_start_date = format_date(start_date)
formatted_end_date = format_date(end_date)

# 키워드
loc = "마포"
food_catecory =  "일식"

# URL 생성
url_layout = f"https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate={formatted_start_date}&endDate={formatted_end_date}&keyword=%23{loc}%20%EB%A7%9B%EC%A7%91%20%23{food_catecory}"

span_selector = "#content > section > div.area_list_search > div > div > div.info_post > div.desc > a.desc_inner > strong > span"
user_agent = "Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/123.0.0.0 Safari/537.36"

print(url_layout)


async def get_page_info(url, session):
    """
    맛집 블로그 정보 개별 page url을 받아서 그 페이지의 내용을 조회 코루틴.
    Return:
        list : 중첩 리스트로 정의.
            한줄의 내용을 리스트로 묶고 그것들 모두 담은 리스트.
    """
    async with session.get(url) as res:
        if res.status == 200:
            html = await res.text()
            soup = BeautifulSoup(html, 'lxml')
            span_list = soup.select(span_selector)

            result_list = [] 
            for sp in span_list:
                sp_list = sp.find_all("td")
                # td는 13개가 있음 / N, 토론방은 사용하지 않을 것이다.
                # tr 구분선은 어떻게 뺄까? 값이 들어가지 않았다.
                # <td colspan="13" class="blank_06"></td> 구분선의 element / 여기서 조건문을 걸 수 있다. 만약 lent(td_list)라면 공백일 걸로 예상이 된다.
                # 조건문은 아래와 같다.
                if len(td_list) == 1:
                    continue #처리하지말고 그다음거 해라 / 구분선 관련된 애들은 이와같은 방법으로 없애라 / 선긋는건 td가 1개 밖에 없다.
                tr_content_list = [] #한행의 cell 정보(td들의 값)들을 담을 리스트.
                for idx, td in enumerate(td_list): #cell 별로 다로 처리하는 것들이 있으므로 enuemrate 사용
                    if idx == 0 or idx == 12: #첫번째(N), 마지막(토론식) 내용은 저장 안함.
                        continue # 저장을 아래서 하니 continue로 그냥 넘겨준다.
                    txt = td.get_text().strip().replace(",", "") # replace로 쉼표 제거 / ""안에 공백이 있으면 안됨 / strip()왜 쓰는거지?
                    if idx == 3: # 전일비. 이미지를 이용해서 상승/하락 조회
                        # 변화 없는 경우, <span class="tah p11">0</span>
                        # 하락인 경우 <img src="https://ssl.pstatic.net/imgstock/images/images4/ico_up.gif" width="7" height="6" style="margin-right:4px;" alt="상승">
                        # 상승이나 하락일 경우, img가 있음 + 상승/하락으로 alt 태그가 뜸/ 이미지가 안뜬다면 알려주는 용도
                        # td 자식으로 이미지 태그 조회. (있으면 상한/하한, 없으면 보합)
                        img = td.find("img")
                        alt_txt = " "
                        if img != None: # 하위에 img 태그가 있으면
                            img_name = img.get("src") #image의 src 속성값
                            if img_name.endswith("ico_up.gif") or img_name.endswith("ico_up2.gif"):# 상승 or 상한가의 element에서 ico_up or up2로 끝나는지에 대한 조건문
                                alt_txt = "+"
                            else:
                                alt_txt = "-"
                        txt = alt_txt + txt
                    tr_content_list.append(txt)
                result_list.append(tr_content_list)
            return result_list




https://section.blog.naver.com/Search/Post.naver?pageNo=1&rangeType=PERIOD&orderBy=sim&startDate=2024-03-31&endDate=2024-03-31&keyword=%23마포%20%EB%A7%9B%EC%A7%91%20%23일식
